<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Building-the-Solution-Design" data-toc-modified-id="Building-the-Solution-Design-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Building the Solution Design</a></span><ul class="toc-item"><li><span><a href="#Cleaning-the-elements" data-toc-modified-id="Cleaning-the-elements-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Cleaning the elements</a></span><ul class="toc-item"><li><span><a href="#Fixing-null-values" data-toc-modified-id="Fixing-null-values-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Fixing null values</a></span></li></ul></li><li><span><a href="#Fixing-metrics-table" data-toc-modified-id="Fixing-metrics-table-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Fixing metrics table</a></span></li><li><span><a href="#Metrics-and-dimensions-SDR" data-toc-modified-id="Metrics-and-dimensions-SDR-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics and dimensions SDR</a></span><ul class="toc-item"><li><span><a href="#Optional-:-Concat-dataframe" data-toc-modified-id="Optional-:-Concat-dataframe-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Optional : Concat dataframe</a></span></li></ul></li><li><span><a href="#Saving-your-file" data-toc-modified-id="Saving-your-file-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Saving your file</a></span></li></ul></li><li><span><a href="#Connecting-with-AEP-Schema" data-toc-modified-id="Connecting-with-AEP-Schema-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Connecting with AEP Schema</a></span><ul class="toc-item"><li><span><a href="#Schema-Manager" data-toc-modified-id="Schema-Manager-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Schema Manager</a></span></li><li><span><a href="#Merging-SDR-with-Schema-definition" data-toc-modified-id="Merging-SDR-with-Schema-definition-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Merging SDR with Schema definition</a></span></li><li><span><a href="#Saving-your-file" data-toc-modified-id="Saving-your-file-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Saving your file</a></span></li></ul></li></ul></div>

With very large organization, it may happen that many people are working on Data Views.\
Your data views are your core representation of your data store.\
It is what is being used for the request by your workspaces and reports.\
Having a clear view on what has been defined in your data view is very important and we can create a script to extract that information very easily in the next steps. 

In order to achieve that view and document it, you can use the cjapy module to build your solution design for your analyst.

We first load cjapy and the configuration used.

In [1]:
import cjapy
cjapy.importConfigFile('myconfig.json')

Once we have done that we can instantiate the connection to CJA API via the `CJA` class.

In [2]:
cja = cjapy.CJA()

# Building the Solution Design

In order to build a solution design, you need to have a complete view of what has been setup in your data view. 

From the `cja` connection that we have built, we will extract the correct data view that we want to see and all of its dimensions and metrics 

In [3]:
dataviews = cja.getDataViews()

In [4]:
dataviews.head(2)

,name,description,owner,isDeleted,parentDataGroupId,segmentList,currentTimezoneOffset,timezoneDesignator,modifiedDate,createdDate,organization,modifiedBy,curationEnabled,recentRecordedAccess,sessionDefinition,externalData,containerNames,parentConnectionId,id,systemUserOwned
0,Luma AJO Data - v1,,{'imsUserId': 'FC00017D578CE2867F000101@AdobeI...,False,dg_d6e144b2-0fba-4557-a144-b20fba3557bc,[],-5.0,US/Eastern,2022-09-13T21:08:10Z,2021-07-15T21:56:53Z,D0F83C645C5E1CC60A495CB3@AdobeOrg,appsvc_admin,true,2025-02-01,"[{'numPeriods': 30, 'granularity': 'minute', '...","{'externalId': 'Luma_AJO_Data___v1', 'external...","{'person': 'Person', 'session': 'Session', 'ev...",dg_d6e144b2-0fba-4557-a144-b20fba3557bc,dv_60f0af258eef5421f3588ecf,NaN
1,Luma AJO Only,,{'imsUserId': 'FC00017D578CE2867F000101@AdobeI...,False,dg_c845c76a-1b33-4056-85c7-6a1b333056c7,[],0.0,GMT,2022-09-13T21:08:15Z,2021-07-30T19:51:43Z,D0F83C645C5E1CC60A495CB3@AdobeOrg,appsvc_admin,true,2025-02-01,"[{'numPeriods': 30, 'granularity': 'minute', '...","{'externalId': 'Luma_AJO_Only', 'externalParen...","{'person': 'Person', 'session': 'Session', 'ev...",dg_c845c76a-1b33-4056-85c7-6a1b333056c7,dv_6104584f3b5f6a6932dcc22d,NaN


Selecting a data view by using its name

In [5]:
dv_id = dataviews.at[dataviews[dataviews['name']=='Adobe Store - Prod'].index[0],'id']
dv_id

'dv_62ba17d5a5d7845496f5fb4d'

Now that we have its id, we can use it to retrieve the different components associated with it

In [6]:
dimensions = cja.getDimensions(dv_id,full=True)
metrics = cja.getMetrics(dv_id,full=True)

The data returned by the methods are `dataframes` which make them very easy to manipulate and to already save them. 

Using the `head` method, you can see the 5 rows that have been returned

In [7]:
dimensions.head()

,id,name,dataSetType,sourceFieldType,baseTableName,type,required,hasData,segmentable,favorite,...,schemaPath,includeExcludeSetting,persistenceSetting,behaviorSetting,substringSetting,sourceFieldId,storageId,multiValued,labels,bucketingSetting
0,variables/daterangesecond,Second,event,standard,hits,time,True,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,variables/65f8fb0a950403085dcdfe0a,Product Super Category,NaN,derived,NaN,string,NaN,True,True,False,...,Derived Fields,{'enabled': False},{'enabled': False},{'lowercase': False},{'enabled': False},NaN,NaN,NaN,NaN,NaN
2,variables/66577f485e767a2c9e40b94d._experience...,Activity Name,lookup,custom,NaN,string,NaN,True,True,False,...,activityName,{'enabled': False},"{'enabled': True, 'allocationModel': {'func': ...",{'lowercase': False},{'enabled': False},66577f485e767a2c9e40b94d._experience.decisioni...,_experience.decisioning.propositions.scopeDeta...,True,"[{'id': 'experimentation_experiment', 'name': ...",NaN
3,variables/adobe_timespentpersession,Time Spent per Session,event,standard,NaN,ordered-enum,NaN,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,variables/66577f485e767a2c9e40b94d._experience...,Active Dates,lookup,custom,NaN,string,NaN,True,True,False,...,activeDates,{'enabled': False},{'enabled': False},{'lowercase': False},{'enabled': False},66577f485e767a2c9e40b94d._experience.decisioni...,_experience.decisioning.propositions.scopeDeta...,True,NaN,NaN


You can see that you have lots of details about each of these dimensions.\
You can see the number of elements by using the `len()` method

In [8]:
len(dimensions)

97

## Cleaning the elements

First of all, you may have default dimensions or metrics that have been integrated in that data view but are not that important because they do not contain data.\
Removing them would actually clean up the table.\
This task, as not sexy, is a crucial task to understand and realize before doing any sort of data analysis in the future.\
We are using this simple task of creating a Solution Design to introduce some concepts such as:
* identifying null values
* cleaning null values

There are some columns that you may also not want to duplicate on your solution design, therefore we will remove them as well.

### Fixing null values

Not a Number (NaN) of Not defined value (na) are elements that could break some basic condition formatting, so we would like to clean these parts first.

In [9]:
dimensions.isnull().sum()

id                         0
name                       0
dataSetType               13
sourceFieldType            0
baseTableName             72
type                       0
required                  78
hasData                    0
segmentable                0
favorite                   0
approved                   0
tags                       0
usageSummary               0
hidden                     0
fromGlobalLookup           0
fieldDefinition            3
derivedFieldCompatible    67
noValueOptionsSetting      0
defaultDimensionSort       0
description               25
sourceFieldName           30
dataSetIds                31
schemaType                30
tableName                 30
hideFromReporting          3
schemaPath                30
includeExcludeSetting     30
persistenceSetting        34
behaviorSetting           33
substringSetting          33
sourceFieldId             43
storageId                 43
multiValued               43
labels                    95
bucketingSetti

In [10]:
dimensions['hasData'] = dimensions['hasData'].fillna(False) ## if no information, just place False as default
dimensions['derivedFieldCompatible'] = dimensions['derivedFieldCompatible'].fillna(False) ## if no information, just place False as default
dimensions['dataSetType'] = dimensions['dataSetType'].fillna("system")  ## if no information, just place "system" as default
dimensions['sourceFieldId'] = dimensions['sourceFieldId'].fillna("cja")

C:\Users\piccini\AppData\Local\Temp\ipykernel_28888\2733145841.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dimensions['derivedFieldCompatible'] = dimensions['derivedFieldCompatible'].fillna(False) ## if no information, just place False as default


Base on these informaiton `hasData`, `derivedFieldCompatible` and `dataSetType`, there could be already a good filtering done for your solution design. 

## Fixing metrics table

You can also look at random lines in your dataframe by using the `sample()` method, the argument giving the number of row to return.  

In [11]:
metrics.sample(4)

,id,name,description,dataSetType,sourceFieldType,baseTableName,type,hideFromReporting,hasData,segmentable,...,schemaType,tableName,schemaPath,includeExcludeSetting,sourceFieldId,storageId,dataSetIds,multiValued,required,attributionSetting
16,metrics/link_clicks,Link Clicks,The quantifiable value of this measure.,event,custom,NaN,decimal,False,True,True,...,double,hits,web.webInteraction.linkClicks.value,{'enabled': False},web.webInteraction.linkClicks.value,explicitw94a6416e5fd321e097a91avalue|hits,[62b2307173a5d61c07a5cb2d],False,NaN,NaN
21,metrics/commerce.purchases.value,Commerce Purchases,The quantifiable value of this measure.,event,custom,NaN,decimal,False,True,True,...,double,hits,commerce.purchases.value,{'enabled': False},commerce.purchases.value,explicitc5667ccad1e55056a8315c4value|hits,"[62b2307173a5d61c07a5cb2d, 631ab2185432701c089...",False,NaN,NaN
12,metrics/qm_api_errors,Quantum Metric API Errors,HTTP 500 error on an API call (ie service has ...,event,custom,NaN,int,False,True,True,...,string,hits,implementationDetails._acxpevangelist.qm_error...,"{'caseSensitive': True, 'match': 'and', 'type'...",implementationDetails._acxpevangelist.qm_error...,expliciti17f67aa7a15fd3c417f853rrorname|hits,[62b2307173a5d61c07a5cb2d],False,NaN,NaN
5,metrics/visits,Sessions,NaN,event,standard,NaN,int,False,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN


In [12]:
metrics.isnull().sum()

id                         0
name                       0
description                6
dataSetType                2
sourceFieldType            0
baseTableName             24
type                       0
hideFromReporting          0
hasData                    0
segmentable                0
favorite                   0
approved                   0
tags                       0
usageSummary               0
hidden                     0
fromGlobalLookup           0
fieldDefinition            4
derivedFieldCompatible    20
sourceFieldName            8
schemaType                 8
tableName                  8
schemaPath                 8
includeExcludeSetting      8
sourceFieldId             10
storageId                 10
dataSetIds                10
multiValued               10
required                  22
attributionSetting        25
dtype: int64

In [13]:
metrics['hasData'] = metrics['hasData'].fillna(False) ## if no information, just place False as default
metrics['derivedFieldCompatible'] = metrics['derivedFieldCompatible'].fillna(False) ## if no information, just place False as default
metrics['dataSetType'] = metrics['dataSetType'].fillna("system")  ## if no information, just place "system" as default
metrics['sourceFieldId'] = metrics['sourceFieldId'].fillna("cja")  ## if no information, just place "cja" as default

C:\Users\piccini\AppData\Local\Temp\ipykernel_28888\2788987130.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  metrics['derivedFieldCompatible'] = metrics['derivedFieldCompatible'].fillna(False) ## if no information, just place False as default


## Metrics and dimensions SDR

The Solution Design Reference basde on CJA implementation can be exported via once we reframe it to the columns we want to have.\
You can filter columns by placing them in a list for filtering.\
If you want to have a copy of your dataframe, use the `copy()` method, that will avoid doing some modification to your original dataframe.

In our example here, we will only select attributes that we find important for the usage of that notebook.

In [14]:
dimensions_sdr = dimensions[dimensions['hasData']][['id','name','dataSetType','sourceFieldId']].copy() ## filtering for dimensions that contain data
metrics_sdr = metrics[metrics['hasData']][['id','name','dataSetType','sourceFieldId']].copy()## filtering for metrics that contain data

### Optional : Concat dataframe

You can combine 2 dataframe via the `concat()` method of the pandas module.

In [15]:
import pandas as pd ## using the pd alias

The `concat` method will take an iterable of dataframe and concat them together.\
`ignore_index` will reset the index.

In [16]:
df_cja = pd.concat([dimensions_sdr,metrics_sdr],ignore_index=True)

In [17]:
df_cja.sample(5)

,id,name,dataSetType,sourceFieldId
95,variables/screen_width,Screen Width,lookup,device.typeID.global-classify-string._globallo...
81,variables/device_screen_orientation,Device Screen Orientation,event,device.screenOrientation
71,variables/first_known_product_name,First Known Product Name,event,productListItems.name
92,variables/implementationDetails._acxpevangelis...,Adobe Journey ID (for QM),event,implementationDetails._acxpevangelist.qm_sessi...
53,variables/6740a8b93c28ee391fec40f2,Next Page Name,system,cja


As you can see the sourcFieldId can be cleaned up as it should provide us with some information about the path used for the data ingestion.\
For the interest of time, we will not cover that part but know that Lookup and profile are ingesting the path with a prefix to avoid collision. 

## Saving your file

You can always save a dataframe as a CSV or as an Excel file.

In [ ]:
df_cja.to_excel('my_sdr.xslx',sheet_name='sdr_combine',index=False) ## to excel

In [ ]:
df_cja.to_csv('my_sdr_schema.csv',index=False) ## to csv

# Connecting with AEP Schema

Customer Journey Analytics is loading the data based on the dataset that are being used in Adobe Experience Platform.\
Getting to know and understand the schema that is being used to capture the data is important.\
In order to do that, you can always log-in to the Adobe Experience Platform via the UI, but you can also retrieve more useful information by using the `aepp` module

The `aepp` module is divided in different services that can be used for analysing your Adobe Experience Platform implementation.\
In our scenario, we will just require to load the `schema` sub module

In [18]:
import aepp
from aepp import schema, schemamanager

The Adobe Experience Platform is divided itself into different sandboxes.\
While loading the configuration, you can specify which sandbox you want to use.\
It is also recommnended to store the configuration in a variable, that we will name `prod` because we are using the prod sandbox.\
We can save the configuratio by passing `True` to the `connectInstance` parameter

In [19]:
prod = aepp.importConfigFile('myconfig.json',sandbox='bighouse',connectInstance=True)

You can then use the configuration to instantiate your connection to your schema for the `prod` sandbox.

In [20]:
schemaProd = schema.Schema(config=prod)

We will retrieve all schemas

In [21]:
allSchemas = schemaProd.getSchemas()

By retrieving the schemas, we have created a storage to easily find the schema ID to be used in these data classes:
* schemaProd.data.schemas_altId
* schemaProd.data.schemas_id

Using the name of our schema, we can easily extract its id: 

In [22]:
schemaProd.data.schemas_id['Adobestore']

'https://ns.adobe.com/acxpevangelist/schemas/988d6949f1a82060f1c658ae1bdd34a19ba0b1e7c436b179'

## Schema Manager

We can use a native functionality of aepp to build a dataframe representation of the schema\
The usage of the `SchemaManager` class will simplify the extraction of the fields

In [23]:
adobeStore = schemamanager.SchemaManager(schemaProd.data.schemas_id['Adobestore'])

In [24]:
df_schema = adobeStore.to_dataframe(queryPath=True,excludeObjects=True)

You can see that the paths have been flatten and provided in 2 columns:
* path : containing the path flatten with more details for list [] and array of objects []{}
* querypath : it is the same path but without the notation that helps understanding its type.
* excludeObjects : If set to True, it will not show the node that serve as object node.

For mapping the path to the one display in CJA, we will use the query path. 

In [25]:
df_schema

,path,type,fieldGroup,title,querypath
0,_id,string,class,Identifier,_id
1,eventMergeId,string,class,ExperienceEvent merge ID,eventMergeId
2,eventType,string,class,Event Type,eventType
4,producedBy,string,class,Produced By,producedBy
5,timestamp,string,class,Timestamp,timestamp
...,...,...,...,...,...
5581,directMarketing.unsubscriptions.value,number,Direct Marketing Details,,directMarketing.unsubscriptions.value
5583,directMarketing.userComplaints.id,string,Direct Marketing Details,Unique Identifier,directMarketing.userComplaints.id
5584,directMarketing.userComplaints.value,number,Direct Marketing Details,,directMarketing.userComplaints.value
5585,experienceevent-bot-detection,None,Bot Detection Information,None,experienceevent-bot-detection


In [27]:
len(df_schema) ## checking the size

4061

## Merging SDR with Schema definition

Once you have your dataframe from the schema manager clean-up you can use it to merge it with the solution design.\
We will create a new dataframe that can replicate a path in case a path is used in both the dimension and the metrics.

In [28]:
from copy import deepcopy

In [29]:
new_dataframe = []
for index, row in df_schema.iterrows():
    data = {}
    flag_found = False
    for index_cja, row_cja in df_cja.iterrows():
        if row['querypath'] in row_cja['id']:
            data['xdm_path'] = row['querypath']
            data['xdm_title'] = row['title']
            data['xdm_type'] = row['type']
            data['cja_id'] = row_cja['id']
            data['cja_name'] = row_cja['name']
            data['cja_type'] = (lambda row : 'dimension' if row['id'].startswith('variables') else 'metric')(row_cja)
            new_dataframe.append(deepcopy(data))
            flag_found = True
            data = {}
    if flag_found == False:
        data['xdm_path'] = row['querypath']
        data['xdm_title'] = row['title']
        data['xdm_type'] = row['type']
        data['cja_id'] = None
        data['cja_name'] = None
        data['cja_type'] = None
        new_dataframe.append(deepcopy(data))

Transforming the new data object into a proper dataframe.

In [30]:
df_new = pd.DataFrame(new_dataframe)

We can show some variables that are defined in the XDM and their corresponding variable name in 

In [31]:
df_new[df_new['cja_id'].astype(bool)].sample(3) ## extract

,xdm_path,xdm_title,xdm_type,cja_id,cja_name,cja_type
2569,device.type,Type,string,variables/device.typeID.global-classify-string...,Audio Support,dimension
2445,_experience.decisioning.propositions,Involved Propositions,array,variables/66577f485e767a2c9e40b94d._experience...,Experience Name,dimension
3480,commerce.purchases.value,,number,metrics/commerce.purchases.value,Commerce Purchases,metric


## Saving your file

You can alsways save your data that is contained in a dataframe by passing the `to_csv()` method

In [ ]:
df_new.to_csv('my_sdr_schema.csv',index=False)